Testing ipyrad functions for speed improvements with numba

In [4]:
import numpy as np
from collections import Counter
from numba import jit

In [136]:
data = [list("AAA-TTTT"),
        list("AAATTTTT"),
        list("AAA-TTTT"),
        list("AAA-TTTN"),
        list("AAA-TTTT"),
        list("AAT-TTCT"),
        list("AAT-TTCT"),
        list("A-T-TTNT"),
        list("AAA-TTCT"),
        list("AAT-TTCT") 
       ]
consens = "AAWNTTYT"
stack = [Counter(i) for i in np.array(data).T]

In [130]:
arr = np.array(data)[:,:]
coldepths = arr.shape[0]
print coldepths

ndepths = np.sum(arr=='N', axis=0)
print ndepths

idepths = np.sum(arr=='-', axis=0)
print idepths

10
[0 0 0 0 0 0 1 1]
[0 1 0 9 0 0 0 0]


In [146]:
A = set(["A", "C"])
B = set(["C", "A"])

A == B

True

In [162]:
np.array([np.sum(arr==x, axis=0) for x in list("CATG")]).T

array([[ 0, 10,  0,  0],
       [ 0,  9,  0,  0],
       [ 0,  6,  4,  0],
       [ 0,  0,  1,  0],
       [ 0,  0, 10,  0],
       [ 0,  0, 10,  0],
       [ 4,  0,  5,  0],
       [ 0,  0,  9,  0]])

In [142]:
hidx = [2,6]
harray = arr[:, [2,6]]

harr = harray[~np.any(harray=="N", axis=1)]
print harr

ccx = Counter([tuple(i) for i in harr])

[i for i in ccx if ccx[i] > 2]

[['A' 'T']
 ['A' 'T']
 ['A' 'T']
 ['A' 'T']
 ['A' 'T']
 ['T' 'C']
 ['T' 'C']
 ['A' 'C']
 ['T' 'C']]


[('T', 'C'), ('A', 'T')]

In [103]:

for row in harray:
    print row

print harray

['A' 'T']
['A' 'T']
['A' 'T']
['A' 'T']
['A' 'T']
['T' 'C']
['T' 'C']
['T' '-']
['A' 'C']
['T' 'C']
[['A' 'T']
 ['A' 'T']
 ['A' 'T']
 ['A' 'T']
 ['A' 'T']
 ['T' 'C']
 ['T' 'C']
 ['T' '-']
 ['A' 'C']
 ['T' 'C']]


In [48]:
for c in stack:
    try:
        c.pop("-")
    except KeyError:
        pass
    try:
        c.pop("N")
    except KeyError:
        pass
print stack

[Counter({'A': 10}), Counter({'A': 9}), Counter({'A': 10}), Counter({'T': 1}), Counter({'T': 10}), Counter({'T': 10}), Counter({'C': 5, 'T': 5}), Counter({'T': 9})]


[Counter({'A': 10}), Counter({'A': 9}), Counter({'A': 10}), Counter({'T': 1}), Counter({'T': 10}), Counter({'T': 10}), Counter({'C': 5, 'T': 5}), Counter({'T': 9})]
[Counter({'A': 9}), Counter({'A': 10}), Counter({'T': 1}), Counter({'T': 10}), Counter({'T': 10}), Counter({'C': 5, 'T': 5}), Counter({'T': 9})]


In [3]:
from collections import Counter
count = [Counter({'T': 22}), Counter({'G': 22}), Counter({'C': 22}), Counter({'A': 22}), Counter({'G': 22}), Counter({'G': 22}), Counter({'G': 22}), Counter({'G': 22}), Counter({'C': 22}), Counter({'A': 22}), Counter({'C': 22}), Counter({'C': 22}), Counter({'C': 22}), Counter({'T': 22}), Counter({'A': 22}), Counter({'T': 21, 'C': 1}), Counter({'C': 22}), Counter({'A': 22}), Counter({'A': 22}), Counter({'C': 21, 'G': 1}), Counter({'A': 22}), Counter({'G': 22}), Counter({'G': 22}), Counter({'G': 22}), Counter({'T': 22}), Counter({'C': 22}), Counter({'T': 22}), Counter({'A': 22}), Counter({'G': 22}), Counter({'C': 22}), Counter({'T': 22}), Counter({'G': 22}), Counter({'C': 22}), Counter({'C': 22}), Counter({'A': 22}), Counter({'A': 22}), Counter({'T': 22}), Counter({'T': 22}), Counter({'C': 22}), Counter({'G': 22}), Counter({'T': 22}), Counter({'G': 22}), Counter({'A': 22}), Counter({'G': 22}), Counter({'G': 22}), Counter({'A': 22}), Counter({'C': 22}), Counter({'T': 22}), Counter({'G': 22}), Counter({'T': 22}), Counter({'T': 22}), Counter({'C': 22}), Counter({'G': 22}), Counter({'T': 22}), Counter({'A': 22}), Counter({'C': 22}), Counter({'C': 22}), Counter({'C': 22}), Counter({'A': 22}), Counter({'A': 22}), Counter({'G': 22}), Counter({'C': 22}), Counter({'T': 22}), Counter({'A': 22}), Counter({'G': 22}), Counter({'A': 22}), Counter({'A': 22}), Counter({'C': 22}), Counter({'A': 22}), Counter({'G': 22}), Counter({'A': 22}), Counter({'A': 22}), Counter({'T': 22}), Counter({'T': 22}), Counter({'C': 22}), Counter({'T': 22}), Counter({'T': 22}), Counter({'T': 22}), Counter({'A': 22}), Counter({'A': 22}), Counter({'T': 22}), Counter({'G': 22}), Counter({'C': 22}), Counter({'A': 22}), Counter({'A': 22}), Counter({'T': 22}), Counter({'A': 22}), Counter({'A': 22}), Counter({'A': 22}), Counter({'T': 22}), Counter({'C': 22}), Counter({'G': 22}), Counter({'T': 22}), Counter({'C': 22})]

count

[Counter({'T': 22}),
 Counter({'G': 22}),
 Counter({'C': 22}),
 Counter({'A': 22}),
 Counter({'G': 22}),
 Counter({'G': 22}),
 Counter({'G': 22}),
 Counter({'G': 22}),
 Counter({'C': 22}),
 Counter({'A': 22}),
 Counter({'C': 22}),
 Counter({'C': 22}),
 Counter({'C': 22}),
 Counter({'T': 22}),
 Counter({'A': 22}),
 Counter({'C': 1, 'T': 21}),
 Counter({'C': 22}),
 Counter({'A': 22}),
 Counter({'A': 22}),
 Counter({'C': 21, 'G': 1}),
 Counter({'A': 22}),
 Counter({'G': 22}),
 Counter({'G': 22}),
 Counter({'G': 22}),
 Counter({'T': 22}),
 Counter({'C': 22}),
 Counter({'T': 22}),
 Counter({'A': 22}),
 Counter({'G': 22}),
 Counter({'C': 22}),
 Counter({'T': 22}),
 Counter({'G': 22}),
 Counter({'C': 22}),
 Counter({'C': 22}),
 Counter({'A': 22}),
 Counter({'A': 22}),
 Counter({'T': 22}),
 Counter({'T': 22}),
 Counter({'C': 22}),
 Counter({'G': 22}),
 Counter({'T': 22}),
 Counter({'G': 22}),
 Counter({'A': 22}),
 Counter({'G': 22}),
 Counter({'G': 22}),
 Counter({'A': 22}),
 Counter({'C': 22}

In [3]:
def get_diploid_lik(starting_params, base_frequencies, stacks, stackcounts):
    """ Log likelihood score given values [H,E] """
    hetero = starting_params[0]
    errors = starting_params[1]
    if (hetero <= 0.) or (errors <= 0.):
        score = np.exp(100)
    else:
        ## get likelihood for all sites
        lik1 = (1.-hetero)*likelihood1(errors, base_frequencies, stacks)
        lik2 = (hetero)*likelihood2(errors, base_frequencies, stacks)
        liks = lik1+lik2
        logliks = np.log(liks[liks > 0])*stackcounts[liks > 0]
        #logliks = np.log(liks)*stackcounts
        score = -logliks.sum()
    return score

In [5]:
@jit
def jget_diploid_lik(starting_params, base_frequencies, stacks, stackcounts):
    """ Log likelihood score given values [H,E] """
    hetero = starting_params[0]
    errors = starting_params[1]
    if (hetero <= 0.) or (errors <= 0.):
        score = np.exp(100)
    else:
        ## get likelihood for all sites
        lik1 = (1.-hetero)*likelihood1(errors, base_frequencies, stacks)
        lik2 = (hetero)*likelihood2(errors, base_frequencies, stacks)
        liks = lik1+lik2
        logliks = np.log(liks[liks > 0])*stackcounts[liks > 0]
        #logliks = np.log(liks)*stackcounts
        score = -logliks.sum()
    return score

In [ ]:
get_diploid_lik

In [2]:
def normtest(x):
    k = 0
    for i in xrange(x):
        k = np.sqrt(2+x)
    return k

In [3]:
%%timeit 
normtest(int(1e6))

1 loops, best of 3: 1.18 s per loop


In [4]:
@jit
def jnormtest(x):
    k = 0
    for i in xrange(x):
        k = np.sqrt(2+x)
    return k

In [5]:
%%time
jnormtest(int(1e6))

CPU times: user 54.3 ms, sys: 12.1 ms, total: 66.4 ms
Wall time: 64.2 ms


1000.0009999995

In [43]:
arr = np.empty([1,20],dtype="S1")

In [44]:
def fillarr(arr, let):
    for i,j in enumerate(let):
        arr[0,i] = j
    return arr

In [45]:
@jit
def jfillarr(arr, let):
    for i,j in enumerate(let):
        arr[0,i] = j
    return arr

### HOly Cow
using jit is so much faster! Just need to rewrite code to fill empty arrays instead of appending to lists and we should be able to use jit just fine for steps 1,2,4,5,7.

In [46]:
%%timeit
fillarr(arr, list("abcdefgh"))

The slowest run took 6.70 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 2.99 µs per loop


In [47]:
%%timeit
jfillarr(arr, list("abcdefgh"))

The slowest run took 778.38 times longer than the fastest. This could mean that an intermediate result is being cached 
1000 loops, best of 3: 209 µs per loop


In [1]:
arr

NameError: name 'arr' is not defined

In [2]:
## pure python
def findbcode(cut, longbar, read1):
    search = read1[1][:longbar+len(cut)]
    countcuts = search.count(cut)
    if countcuts == 1:
        barcode = search.split(cut, 1)[0]
    elif countcuts == 2:
        barcode = search.rsplit(cut, 2)[0]
    else:
        barcode = ""
    return barcode

In [42]:
## jit version
@jit
def jfindbarcode(cut, longbar, read1):
    search = read1[1][:longbar+len(cut)]
    countcuts = search.count(cut)
    if countcuts == 1:
        barcode = search.split(cut, 1)[0]
    elif countcuts == 2:
        barcode = search.rsplit(cut, 2)[0]
    else:
        barcode = ""
    return barcode

In [47]:
cut = "TGCAG"
longbar = 6
read1 = ['fakeread','AAACCCTGCAGAAAAAAAAAAAAAAAAA']
nread1 = np.array(['fakeread','AAACCCTGCAGAAAAAAAAAAAAAAAAA'])


In [48]:
%%timeit 
findbcode(cut, longbar, read1)

The slowest run took 5.48 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 914 ns per loop


In [59]:
%%timeit
findbcode(cut, longbar, nread1)

The slowest run took 11.24 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 1.08 µs per loop


In [51]:
%%timeit 
jfindbarcode(cut, longbar, nread1)

10000 loops, best of 3: 53.5 µs per loop


In [37]:
read1[1][:longbar+6].count("TGCAG")

1